# Fuzzy CoCo - Fuzzy Modelling for Explainable Artificial Intelligence



Fuzzycocopython is under developpment and is accessible on request / repo has been made public here: TODO / is currently not publicly available. 

Your first need to clone the repository by running `git clone asdfasdf`

IMPTOX STUFF

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold
import kagglehub

To import fuzzycocopython, we need to be in the cloned git repository. For this, replace the path in the cd command bellow to adjust the path to where you clone the fuzzycocopython repository.

In [2]:
wd = !pwd  # Execute `pwd` and capture the output as a Python list - current working is in wd[0]
print(f"Current working directory: {wd[0]}")  # Print the working directory


# For the particular case of fuzzycocopython, once the repository is cloned switch directory for import and come back to the original working directory
%cd /mnt/c/Users/TSchowing/Desktop/repositories/fuzzycocopython/

import fuzzycocopython

#%cd {wd[0]}


Current working directory: /mnt/c/Users/TSchowing/Desktop/repositories/IMPTOX_XAI
/mnt/c/Users/TSchowing/Desktop/repositories/fuzzycocopython


/mnt/c/Users/TSchowing/Desktop/repositories/IMPTOX_XAI/.venvfuzzycoco/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


Here we load the data. For this example we will use the pima indian diabetes dataset. 

In [3]:
# Download latest version
path = kagglehub.dataset_download("gargmanas/pima-indians-diabetes")

print("Path to dataset files:", path )

df = pd.read_csv(path + '/pima-indians-diabetes.csv')
df.columns = ['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction','Age','OUT']

X = df.drop('OUT', axis=1)
y = df['OUT']

# Just to confirm that X is a DataFrame
if isinstance(X, pd.DataFrame):
    print("Yes, X is a DataFrame")
else:
    raise ValueError("X is not a DataFrame")

Path to dataset files: /home/twg/.cache/kagglehub/datasets/gargmanas/pima-indians-diabetes/versions/1
Yes, X is a DataFrame


We can directly define an evolution (a model) with certain parameters. However, the power of fuzzycocopython is that it is compatible with scikit-learn which allows to perform grid search to find the best hyper-parameters and fit different models. 

In [8]:


#model = fuzzycocopython.FuzzyCocoClassifier({'maxGenPop1': 500, 'accuracyW': 0.25, 'dontCareW': 0.05, 'ppvW': 0.5, 'rmseW': 0.25, 'sensitivityW': 1.0, 'specificityW': 1.0, 'cxProbPop1': 0.7, 'cxProbPop2': 0.8, 'elitePop1': 6, 'elitePop2': 6, 'popSizePop1': 250, 'popSizePop2': 250})

# Define parameter grids to choose from (just examples)
fuzzy_system = {
    "nbRules": [2, 3, 7],
    "nbMaxVarPerRule": [2, 3, 4],
    "nbInSets": [2, 3],
    "nbOutSets": [2, 3],
}

fitness_parameters = {
    "sensitivityW": [0.5, 1.0],
    "specificityW": [0.5, 1.0],
    "accuracyW": [0.25, 0.75],
    "ppvW": [0.25, 0.5, 0.75],
    "rmseW": [0.25, 0.5, 0.75],
    "dontCareW": [0.05, 0.1]
}

co_evolution = {
    "popSizePop1": [250, 500],
    "popSizePop2": [250, 500],
    "cxProbPop1": [0.7, 0.9],
    "elitePop1": [6, 12],
    "elitePop2": [6, 12],
    "cxProbPop2": [0.6, 0.8],
}



# Combine all parameter grids
grid = {
    "fuzzy_system": fuzzy_system,
    "fitness_parameters": fitness_parameters,
    "co_evolution": co_evolution,
}

# Repeated Stratified K-Fold (n_splits=4, controlled by n_repeats)
rskf = RepeatedStratifiedKFold(
    n_splits=4, 
    n_repeats=3, 
    random_state=42  # You can change this seed if desired
)

param_grid = grid['fuzzy_system']


model = fuzzycocopython.FuzzyCocoClassifier()

gs = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    cv=rskf,
    n_jobs=1,
    scoring='accuracy',
    verbose=3,
    return_train_score=True
)

# Fit the grid search
gs.fit(X, y)

# Export results to a DataFrame
results = pd.DataFrame(gs.cv_results_)
#results.to_csv(output_path, index=False)



| nb_rules | nb_max_var_per_rule | max_generations | max_fitness | nb_cooperators |
|----------|---------------------|-----------------|-------------|----------------|
        2 |                   2 |             100 |           0 |              2 |

Input Variables:
| nb_sets | nb_bits_vars | nb_bits_sets | nb_bit_pos |
|---------|--------------|--------------|------------|
       2 |            7 |            2 |          8 |

Output Variables:
| nb_sets | nb_bits_vars | nb_bits_sets | nb_bit_pos |
|---------|--------------|--------------|------------|
       2 |            1 |            1 |          1 |

defuzz threshold:ACTIVATED
output_vars_defuzz_thresholds:{0.5}
Rules params
| pop_size | elite_size | cx_prob | mut_flip_genome | mut_flip_bit |
|----------|------------|---------|-----------------|--------------|
      350 |         10 |       0 |               0 |            0 |

MFs (Membership Functions) params
| pop_size | elite_size | cx_prob | mut_flip_genome | mut_flip_bit

[CV 4/12] END nbInSets=2, nbMaxVarPerRule=2, nbOutSets=2, nbRules=2;, score=(train=0.655, test=0.660) total time=  26.9s


##========================================================================
##FuzzyCocoScriptRunner::run()
## params: Fuzzy Coco Params:
Global params:
| nb_rules | nb_max_var_per_rule | max_generations | max_fitness | nb_cooperators |
|----------|---------------------|-----------------|-------------|----------------|
        2 |                   2 |             100 |           0 |              2 |

Input Variables:
| nb_sets | nb_bits_vars | nb_bits_sets | nb_bit_pos |
|---------|--------------|--------------|------------|
       2 |            7 |            2 |          8 |

Output Variables:
| nb_sets | nb_bits_vars | nb_bits_sets | nb_bit_pos |
|---------|--------------|--------------|------------|
       2 |            1 |            1 |          1 |

defuzz threshold:ACTIVATED
output_vars_defuzz_thresholds:{0.5}
Rules params
| pop_size | elite_size | cx_prob | mut_flip_genome | mut_flip_bit |
|----------|------------|---------|-----------------|--------------|
      350 |       

[CV 5/12] END nbInSets=2, nbMaxVarPerRule=2, nbOutSets=2, nbRules=2;, score=(train=0.696, test=0.656) total time=  27.1s


##========================================================================
##FuzzyCocoScriptRunner::run()
## params: Fuzzy Coco Params:
Global params:
| nb_rules | nb_max_var_per_rule | max_generations | max_fitness | nb_cooperators |
|----------|---------------------|-----------------|-------------|----------------|
        2 |                   2 |             100 |           0 |              2 |

Input Variables:
| nb_sets | nb_bits_vars | nb_bits_sets | nb_bit_pos |
|---------|--------------|--------------|------------|
       2 |            7 |            2 |          8 |

Output Variables:
| nb_sets | nb_bits_vars | nb_bits_sets | nb_bit_pos |
|---------|--------------|--------------|------------|
       2 |            1 |            1 |          1 |

defuzz threshold:ACTIVATED
output_vars_defuzz_thresholds:{0.5}
Rules params
| pop_size | elite_size | cx_prob | mut_flip_genome | mut_flip_bit |
|----------|------------|---------|-----------------|--------------|
      350 |       

...

##========================================================================
##FuzzyCocoScriptRunner::run()
## params: Fuzzy Coco Params:
Global params:
| nb_rules | nb_max_var_per_rule | max_generations | max_fitness | nb_cooperators |
|----------|---------------------|-----------------|-------------|----------------|
        7 |                   4 |             100 |           0 |              2 |

Input Variables:
| nb_sets | nb_bits_vars | nb_bits_sets | nb_bit_pos |
|---------|--------------|--------------|------------|
       3 |            7 |            2 |          8 |

Output Variables:
| nb_sets | nb_bits_vars | nb_bits_sets | nb_bit_pos |
|---------|--------------|--------------|------------|
       3 |            1 |            1 |          1 |

defuzz threshold:ACTIVATED
output_vars_defuzz_thresholds:{0.5}
Rules params
| pop_size | elite_size | cx_prob | mut_flip_genome | mut_flip_bit |
|----------|------------|---------|-----------------|--------------|
      350 |       

[CV 11/12] END nbInSets=3, nbMaxVarPerRule=4, nbOutSets=3, nbRules=7;, score=(train=0.682, test=0.672) total time= 1.7min


##========================================================================
##FuzzyCocoScriptRunner::run()
## params: Fuzzy Coco Params:
Global params:
| nb_rules | nb_max_var_per_rule | max_generations | max_fitness | nb_cooperators |
|----------|---------------------|-----------------|-------------|----------------|
        7 |                   4 |             100 |           0 |              2 |

Input Variables:
| nb_sets | nb_bits_vars | nb_bits_sets | nb_bit_pos |
|---------|--------------|--------------|------------|
       3 |            7 |            2 |          8 |

Output Variables:
| nb_sets | nb_bits_vars | nb_bits_sets | nb_bit_pos |
|---------|--------------|--------------|------------|
       3 |            1 |            1 |          1 |

defuzz threshold:ACTIVATED
output_vars_defuzz_thresholds:{0.5}
Rules params
| pop_size | elite_size | cx_prob | mut_flip_genome | mut_flip_bit |
|----------|------------|---------|-----------------|--------------|
      350 |       

[CV 12/12] END nbInSets=3, nbMaxVarPerRule=4, nbOutSets=3, nbRules=7;, score=(train=0.663, test=0.696) total time=  59.6s


nb_cooperators |
|----------|---------------------|-----------------|-------------|----------------|
        2 |                   3 |             100 |           0 |              2 |

Input Variables:
| nb_sets | nb_bits_vars | nb_bits_sets | nb_bit_pos |
|---------|--------------|--------------|------------|
       2 |            7 |            2 |          8 |

Output Variables:
| nb_sets | nb_bits_vars | nb_bits_sets | nb_bit_pos |
|---------|--------------|--------------|------------|
       3 |            1 |            1 |          1 |

defuzz threshold:ACTIVATED
output_vars_defuzz_thresholds:{0.5}
Rules params
| pop_size | elite_size | cx_prob | mut_flip_genome | mut_flip_bit |
|----------|------------|---------|-----------------|--------------|
      350 |         10 |       0 |               0 |            0 |

MFs (Membership Functions) params
| pop_size | elite_size | cx_prob | mut_flip_genome | mut_flip_bit |
|----------|------------|---------|-----------------|-----------

In [9]:
results.to_csv("./results_fuzzy.csv", index=False)

In [10]:
results = pd.read_csv("./results_fuzzy.csv")
results.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_nbInSets,param_nbMaxVarPerRule,param_nbOutSets,param_nbRules,params,split0_test_score,...,split4_train_score,split5_train_score,split6_train_score,split7_train_score,split8_train_score,split9_train_score,split10_train_score,split11_train_score,mean_train_score,std_train_score
0,22.884210,6.326626,0.003727,0.001657,2,2,2,2,"{'nbInSets': 2, 'nbMaxVarPerRule': 2, 'nbOutSe...",0.708333,...,0.695652,0.711304,0.714783,0.649306,0.714783,0.680000,0.753043,0.652778,0.696383,0.033254
1,33.508881,10.936910,0.003317,0.000324,2,2,2,3,"{'nbInSets': 2, 'nbMaxVarPerRule': 2, 'nbOutSe...",0.656250,...,0.693913,0.756522,0.650435,0.668403,0.674783,0.685217,0.725217,0.699653,0.682171,0.030953
2,45.985066,21.395406,0.003641,0.000510,2,2,2,7,"{'nbInSets': 2, 'nbMaxVarPerRule': 2, 'nbOutSe...",0.619792,...,0.688696,0.693913,0.688696,0.706597,0.650435,0.652174,0.680000,0.652778,0.670433,0.019180
3,25.827819,1.808530,0.003404,0.000295,2,2,3,2,"{'nbInSets': 2, 'nbMaxVarPerRule': 2, 'nbOutSe...",0.682292,...,0.664348,0.681739,0.660870,0.703125,0.713043,0.739130,0.718261,0.657986,0.693036,0.027788
4,34.298507,10.949415,0.003447,0.000533,2,2,3,3,"{'nbInSets': 2, 'nbMaxVarPerRule': 2, 'nbOutSe...",0.656250,...,0.676522,0.662609,0.721739,0.703125,0.671304,0.653913,0.721739,0.685764,0.693611,0.027568


In [3]:
# Print best parameters and score
print("Best Parameters:", gs.best_params_)
print("Best Score:", gs.best_score_)

NameError: name 'gs' is not defined

In [6]:
!pip show fuzzycocopython

Name: fuzzycocopython
Version: 0.0.1
Summary: Python API for FuzzyCoco using C++ bindings
Home-page: 
Author: 
Author-email: Arthur Babey <arthur.babey@heig-vd.ch>
License: 
Location: /mnt/c/Users/TSchowing/Desktop/repositories/IMPTOX_XAI/.venvfuzzycoco/lib/python3.12/site-packages
Requires: numpy, pandas, pre-commit, pybind11, pybind11-global, pytest, scikit-learn, scikit_build_core
Required-by: 


In [7]:
!ls /mnt/c/Users/TSchowing/Desktop/repositories/IMPTOX_XAI/.venvfuzzycoco/lib/python3.12/site-packages/fuzzycocopython/


__init__.py  demo-notebook.ipynb  fuzzycoco_classifier.py  fuzzysystem.ffs
__pycache__  fuzzycoco_base.py	  fuzzycoco_regressor.py   utils.py
